##### 0. 스페셜 토큰 처리?

In [1]:
# 분할 x 단어
# not_tokenized = ['#Address#', '#CarNumber#', '#CardNumber#', '#DateOfBirth#', '#Email#', 
#                 '#PassportNumber#', '#Person#', '#Person1#', '#Person2#', '#Person3#', 
#                 '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#PhoneNumber#', '#SSN#']

##### 1. 발화자 / 대화 분리해서 학습

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import wordnet
import random

# NLTK 데이터 다운로드 (최초 1회만 필요)
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
def get_wordnet_pos(word):
    """단어의 품사를 반환하는 함수"""
    tag = nltk.pos_tag([word])[0][1]
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag[0], wordnet.NOUN)

def synonym_replacement(words, n=1):
    """동의어로 단어 대체"""
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    
    new_words = words.copy()
    random_word_list = list(set([word for word in words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = []
        try:
            for syn in wordnet.synsets(random_word):
                for lemma in syn.lemmas():
                    # 동의어 추가 (원본 단어와 다른 것만)
                    if lemma.name().lower() != random_word.lower():
                        synonyms.append(lemma.name())
        except:
            continue
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(set(synonyms)))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        
        if num_replaced >= n:
            break
    
    return new_words

def random_insertion(words, n=1):
    """임의의 동의어 삽입"""
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    
    new_words = words.copy()
    for _ in range(n):
        # 랜덤 단어 선택
        if len(new_words) > 0:
            random_word = random.choice(new_words)
            synonyms = []
            try:
                for syn in wordnet.synsets(random_word):
                    for lemma in syn.lemmas():
                        if lemma.name().lower() != random_word.lower():
                            synonyms.append(lemma.name())
            except:
                continue
            
            if synonyms:
                synonym = random.choice(list(set(synonyms)))
                random_index = random.randint(0, len(new_words))
                new_words.insert(random_index, synonym)
    
    return new_words

def random_swap(words, n=1):
    """단어 위치 무작위 교환"""
    new_words = words.copy()
    for _ in range(n):
        if len(new_words) >= 2:
            idx1, idx2 = random.sample(range(len(new_words)), 2)
            new_words[idx1], new_words[idx2] = new_words[idx2], new_words[idx1]
    return new_words

def random_deletion(words, p=0.1):
    """단어 무작위 삭제"""
    if len(words) == 0:
        return words
    
    new_words = [word for word in words if random.random() > p]
    
    # 모든 단어가 삭제되는 것을 방지
    if len(new_words) == 0:
        new_words = [random.choice(words)]
    
    return new_words

def augment_text_data_with_EDA(text, repetition=1):
    """입력된 문장에 대해 EDA를 통해 데이터 증강"""
    words = text.split()
    augmented_texts = []
    
    for _ in range(repetition):
        # 다양한 증강 기법 적용
        augmented_list = []
        
        # 동의어 대체
        augmented_list.append(' '.join(synonym_replacement(words)))
        
        # 랜덤 삽입
        augmented_list.append(' '.join(random_insertion(words)))
        
        # 랜덤 스왑
        augmented_list.append(' '.join(random_swap(words)))
        
        # 랜덤 삭제
        augmented_list.append(' '.join(random_deletion(words)))
        
        augmented_texts.extend(augmented_list)
    
    return augmented_texts

In [4]:
# 화자와 대화 분리 함수
def split_speaker_and_text(dialogue):
    """대화 내용을 화자 태그와 분리"""
    pattern = r'(#Person\d+#): (.+)'
    lines = dialogue.split('\n')
    return [(match.group(1), match.group(2)) for line in lines if (match := re.match(pattern, line))]

# 화자와 대화 결합 함수
def join_speaker_and_text(speaker_text_pairs):
    """화자 태그와 대화 내용을 결합"""
    return '\n'.join([f'{speaker}: {text}' for speaker, text in speaker_text_pairs])

In [5]:
# 데이터 로드
df = pd.read_csv('/root/NLP/data/train.csv')
df_filtered = df[['fname', 'dialogue', 'summary', 'topic']]

# 증강 데이터를 저장할 리스트
augmented_rows = []

# EDA 적용 및 데이터 증강
for index, row in df_filtered.iterrows():
    original_text = row['dialogue']

    # 화자 태그와 대화 내용 분리
    speaker_text_pairs = split_speaker_and_text(original_text)

    # 대화 내용에 EDA 적용
    augmented_text_lists = [
        augment_text_data_with_EDA(text, repetition=3) for _, text in speaker_text_pairs
    ]

    # EDA 결과와 화자 태그 결합
    for augmented_texts in zip(*augmented_text_lists):
        new_speaker_text_pairs = [
            (speaker_text_pairs[i][0], augmented_text) 
            for i, augmented_text in enumerate(augmented_texts)
        ]
        augmented_dialogue = join_speaker_and_text(new_speaker_text_pairs)

        # 증강 데이터를 새로운 행으로 추가
        augmented_rows.append({
            'fname': row['fname'],
            'dialogue': augmented_dialogue,
            'summary': row['summary'],
            'topic': row['topic']
        })

    # 진행 상황 출력
    if index % 100 == 0:
        print(f"Processed {index + 1} rows out of {len(df_filtered)}")

# 증강 데이터를 DataFrame으로 변환
augmented_data = pd.DataFrame(augmented_rows)

# 결과 저장
augmented_data.to_csv('/root/NLP/data/aug_nltk_train.csv', index=False)

Processed 1 rows out of 12457
Processed 101 rows out of 12457
Processed 201 rows out of 12457
Processed 301 rows out of 12457
Processed 401 rows out of 12457
Processed 501 rows out of 12457
Processed 601 rows out of 12457
Processed 701 rows out of 12457
Processed 801 rows out of 12457
Processed 901 rows out of 12457
Processed 1001 rows out of 12457
Processed 1101 rows out of 12457
Processed 1201 rows out of 12457
Processed 1301 rows out of 12457
Processed 1401 rows out of 12457
Processed 1501 rows out of 12457
Processed 1601 rows out of 12457
Processed 1701 rows out of 12457
Processed 1801 rows out of 12457
Processed 1901 rows out of 12457
Processed 2001 rows out of 12457
Processed 2101 rows out of 12457
Processed 2201 rows out of 12457
Processed 2301 rows out of 12457
Processed 2401 rows out of 12457
Processed 2501 rows out of 12457
Processed 2601 rows out of 12457
Processed 2701 rows out of 12457
Processed 2801 rows out of 12457
Processed 2901 rows out of 12457
Processed 3001 rows ou

In [6]:
augmented_data

,fname,dialogue,summary,topic
0,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
1,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
2,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 의사입니다. 호킨스 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
3,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
4,train_0,"#Person1#: 안녕하세요, 스미스씨. 저는 호킨스 의사입니다. 오늘 왜 오셨나...","스미스씨가 건강검진을 받고 있고, 호킨스 의사는 매년 건강검진을 받는 것을 권장합니...",건강검진 받기
...,...,...,...,...
149479,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기
149480,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기
149481,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기
149482,train_12459,"#Person1#: 엄마, 다음 가족을 이 삼촌네 토요일에 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


##### 2. df + augmented_data concat하기

##### 3. 인덱스 reset / fname 열 바꾸기(train_0 ~ train?)